In [18]:
# Load libraries
import numpy as np
import pandas as pd
import urllib.request, json
import datetime as dt
from sklearn.ensemble import RandomForestRegressor
#from functools import reduce
#import matplotlib.pyplot as plt
#from sklearn.feature_selection import SelectKBest
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import accuracy_score, confusion_matrix
#from sklearn import svm
#from sklearn.svm import SVC
#from sklearn import metrics

In [2]:
## Inputs

# Ticker
ticker = 'AAPL'
historical_days = 2000
days_ahead_to_predict = 1 # if this changes, adjust logic below that calculates % change

# API Key for Alpha Vantage
api_key = 'A3Y10PJZ4M4FYKSW'

In [5]:
# Get daily adjusted stock price data
url_string_adjdaily = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&outputsize=full&apikey=%s'%(ticker, api_key)
with urllib.request.urlopen(url_string_adjdaily) as url:
    data = json.loads(url.read().decode())
    # Extract data
    data = data['Time Series (Daily)']
    df_adjclose = pd.DataFrame(columns=['Date','AdjClose'])
    for k,v in data.items():
        date = dt.datetime.strptime(k, '%Y-%m-%d')
        data_row = [date.date(), float(v['5. adjusted close'])]
        df_adjclose.loc[-1,:] = data_row
        df_adjclose.index = df_adjclose.index + 1

In [6]:
# Take last x number of data points
df_adjclose = df_adjclose.head(historical_days)

# Drop date variable
df_final = df_adjclose.drop(['Date'], 1)

# Reverse the dataframe to get data in chronological order
df_final = df_final.iloc[::-1]

In [13]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


# load the dataset
dataframe = df_final['AdjClose']
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset = dataset.reshape(-1, 1)

In [14]:
dataset

array([[ 50.5887],
       [ 49.5913],
       [ 49.9441],
       ...,
       [218.96  ],
       [220.82  ],
       [227.01  ]], dtype=float32)

In [15]:
look_back = 100
trainX, trainY = create_dataset(dataset, look_back)

In [19]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(trainX, trainY)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [22]:
# Save model file
from sklearn.externals import joblib
filename = 'RFmodel_AAPL.sav'
joblib.dump(regressor, filename)

/misc/DLshare/home/rpcdz397/Projects/Marketing_Optimization/LTVModel/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['RFmodel_AAPL.sav']

In [23]:
# Get daily adjusted stock price data
url_string_adjdaily = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&outputsize=full&apikey=%s'%(ticker, api_key)
with urllib.request.urlopen(url_string_adjdaily) as url:
    data = json.loads(url.read().decode())
    # Extract data
    data = data['Time Series (Daily)']
    df_adjclose = pd.DataFrame(columns=['Date','AdjClose'])
    for k,v in data.items():
        date = dt.datetime.strptime(k, '%Y-%m-%d')
        data_row = [date.date(), float(v['5. adjusted close'])]
        df_adjclose.loc[-1,:] = data_row
        df_adjclose.index = df_adjclose.index + 1

historical_days = 100
# Take last x number of data points
df_adjclose_pred = df_adjclose.head(historical_days)

# Drop date variable
df_final_pred = df_adjclose_pred.drop(['Date'], 1)

# Reverse the dataframe to get data in chronological order
df_final_pred = df_final_pred.iloc[::-1]

In [24]:
# load the dataset
dataframe_pred = df_final_pred['AdjClose']
dataset_pred = dataframe_pred.values
dataset_pred = dataset_pred.astype('float32')
dataset_pred = dataset_pred.reshape(1, -1)

# load the model from disk
loaded_model = joblib.load(filename)

# Make prediction
#pred_data = numpy.reshape(dataset_pred, (dataset_pred.shape[0], 1, dataset_pred.shape[1]))


In [25]:
pred = loaded_model.predict(dataset_pred)
print(pred)

[220.51719666]
